**1. Optimization practice** this is just a review of the code we have for solving Bellman equations. 

In [1]:
using LinearAlgebra, Interpolations, Distributions, FastGaussQuadrature
Φ(x::T) where {T<:Real} = cdf(Normal(0, 1), x)


function rouwenhorst(N::Integer,ρ::T1,σ::T2,μ = zero(promote_type(T1, T2))) where {T1<:Real,T2<:Real}
    σ_y = σ / sqrt(1-ρ^2)
    p   = (1+ρ)/2
    Θ   = [p 1-p; 1-p p]
    Δ   = sqrt(N-1) * σ_y
    Y,P = _rouwenhorst(p, p, μ, Δ, N)
    return Y,P
end

function _rouwenhorst(p::Real, q::Real, μ::Real, Δ::Real, N::Integer)
    if N == 2
        return [μ-Δ,μ+Δ],[p 1-p;1-q q]
    else
        _, P_nm1       = _rouwenhorst(p,q,μ,Δ,N-1)
        P              = p*[P_nm1 zeros(N-1,1); zeros(1,N)]+
                         (1-p)*[zeros(N-1,1) P_nm1; zeros(1,N)] +
                         q*[zeros(1,N); zeros(N-1,1) P_nm1] +
                         (1-q)*[zeros(1,N); P_nm1 zeros(N-1,1)]
        P[2:end-1,:] ./= 2
        Y              = collect(range(μ-Δ, stop=μ+Δ, length=N))
        return Y,P
    end
end

_rouwenhorst (generic function with 1 method)

In [2]:

function expectation(x,P)
    return x*P'
end

function t_operator(v,kgrid,agrid,param,P,Ri) 
    """
    Defines the T operator as the list of operations to solve the maximization program
    in the right hand side of the Bellman equation: max u(c_t)+β v(k_{t+1})
    
    Input:
      v:     The guess of the value function at current iteration 
      kgrid: The capital grid
      param: A (4x1) vector of parameters
      agrid: Grid of values for the shock
      P    : Transition matrix for the shock
    
    Output: 
     Tv: The new guess for the value function (we applied the T operator to v)
     Gk: The implied optimal capital decision rule
    """   


    α,β,δ,σ = param
    Tv   = similar(v)
    Gk   = similar(v)    
    Nk,Na= size(v) ##this is symmetric, two grid points we may have.
    Ev   = expectation(v,P)

  for ia in 1:Na
        next_index=1
        for ik in 1:Nk
            Vmax    =-Inf
            k_prime = 0.0
            r       = Ri[ik,ia]
            for jk in next_index:Nk
                kp   = kgrid[jk]
                vtmp = util(r-kp,σ)+β*Ev[jk,ia]
                if vtmp>=Vmax
                    next_index= jk
                    Vmax      = vtmp
                    k_prime   = kp
                else
                    break
                end
            end
            @inbounds Tv[ik,ia] = Vmax
            @inbounds Gk[ik,ia] = k_prime
        end 
    end
    return Tv,Gk
end



t_operator (generic function with 1 method)

**Step2** I need to create the bellman equation loop. The idea is to loop over it to find a fixed point. 

In [3]:
function bellman_equation(kgrid,agrid,param,P,t_operator,Ri;criterion=:value)
    tol=1e-08
    crit=1
    Nk = length(kgrid)
    Na = length(agrid)
    v  = zeros(Float64,Nk,Na) 
    Gk0= similar(v)

    while err>tol
        
        Tv, Gv= t_operator(v,kgrid,agrid,param,P,Ri)
        crit   = criterion== :value ? maximum(abs.(Tv-v)) : maximum(abs(Gv-Gk0))##this is important to keep in mind.
        v= copy(Tv)
        Gk0= copy(Gv)

    end 
end 

bellman_equation (generic function with 1 method)

In [4]:
# Define the parametric assumptions

# 1) Define some functions
util = (c,σ) -> c>0 ? (c^(1-σ)-1.0)/(1.0-σ) : -Inf  # Utility function, taking care of negative values of x
R    = (k,a,α,δ) -> a*k^α+(1.0-δ)*k                 # Total available resources

# 2) Define some parameters
α    = 0.33    # Capital elasticity in production function
β    = 0.96    # Discount factor
δ    = 0.10    # Depreciation rate
σ    = 2.00    # utility curvature (Intertemporal eleasticity of substitution)
param= [α,β,δ,σ]

# 3) Define the shocks -> log(a_t) = ρ \log(a_{t-1}) + (1-ρ)μ + ϵ_t  ϵ_t ~> N(0,σ²)  
# and approximate it by a Markov Chain
ρ    = 0.8     # persistence of the shock
σ    = 0.02    # Volatility of the innovation
μ    = 0       # Average of the process
N    = 5       # Number of points in the grid of shocks
# Δ    = 3.0     # number of std deviations away from the mean (Tauchen only)
# la,P = tauchen(N,ρ,σ,μ,Δ)   # If Tauchen's (1986) method is chosen
la,P = rouwenhorst(N,ρ,σ,μ) # If Rouwenhorst's (1995) method is chosen
# p=(1.0+ρ)/2.0
# pi=0.02
# da=0.025
# disast=0.2
# la=[da -da log(1-disast)-da]
# P=[(1-pi)*p (1-pi)*(1-p) pi;(1-pi)*(1-p) (1-pi)*p pi;0.5 0.5 0]
# la   = [-0.05 0.05]
# P    = [0.8 0.2;0.2 0.8]
# N    = 2
agrid= exp.(la);

# 4) Define the grid
kss  = (α*β/(1.0-β*(1.0-δ)))^(1.0/(1.0-α)) # Deterministic steady state of capital
Δk   = 0.5
kmin = (1.0-Δk)*kss
kmax = (1.0+Δk)*kss
Nk   = 1_000
kgrid= range(kmin,stop=kmax,length=Nk)
Ri   = [R(kgrid[ik],agrid[ia],α,δ) for ik in 1:Nk,ia in 1:N]
;

In [5]:
##Apply the function we have just coded... Probably it will not work 
@time v,Gk=bellman_equation(kgrid,agrid,param,P,t_operator,Ri;criterion=:decision_rule)



MethodError: MethodError: no method matching isless(::Float64, ::Nothing)
The function `isless` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  isless(!Matched::Missing, ::Any)
   @ Base missing.jl:87
  isless(::Any, !Matched::Missing)
   @ Base missing.jl:88
  isless(::AbstractFloat, !Matched::Union{StatsBase.PValue, StatsBase.TestStat})
   @ StatsBase ~/.julia/packages/StatsBase/yZS2U/src/statmodels.jl:100
  ...
